In [2]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('./AQI/2020-2022_history_data.csv')
df.tail()

,Station,Date,Time,latitude,longitude,PM25,PM10,CO,NO,NO2,NH3,O3,SO2
1604659,"Anand Vihar, Delhi, Delhi, India",02-09-2022,14:00:00,28.6459,77.3149,215.49,317.42,1922.61,0.40,54.15,17.48,20.39,24.56
1604660,"Anand Vihar, Delhi, Delhi, India",02-09-2022,15:00:00,28.6459,77.3149,233.83,362.83,2189.64,3.52,53.47,16.72,4.25,25.27
1604661,"Anand Vihar, Delhi, Delhi, India",02-09-2022,16:00:00,28.6459,77.3149,263.07,421.80,2563.48,12.18,47.30,16.97,0.43,25.51
1604662,"Anand Vihar, Delhi, Delhi, India",02-09-2022,17:00:00,28.6459,77.3149,296.97,481.72,2964.02,23.25,41.13,17.99,0.02,25.75
1604663,"Anand Vihar, Delhi, Delhi, India",02-09-2022,18:00:00,28.6459,77.3149,305.72,490.28,3044.13,29.95,36.67,16.72,0.00,22.65


In [4]:
df.drop(columns=['NO'],inplace=True)

#### All the Concentration in the dataframe are in ug/m3.
Converting them into respective Values as shown below.

* PM2.5 is measured in ug / m3 
* PM10 is measured in ug / m3 
* SO2 is measured in ug / m3 
* NOx is measured in ppb
* NH3 is measured in ug / m3 
* CO is measured in mg / m3
* O3 is measured in ug / m3

In [5]:
df['Date']= pd.to_datetime(df['Date'], infer_datetime_format=True)
df['Date'] = df['Date'].astype(str)
df.sort_values(['Station','Date','Time'],inplace=True)

df['CO'] = df['CO']/1000            # Converting CO concentration from ug/m3 to mg/m3
df['NO2'] = df['NO2']*24.45/46.01   # convertion NO2 concentration from mg/m3 to ppb

#### Calculation of Rolling average and max values

In [6]:
df["PM10_24hr_avg"] = df.groupby("Station")["PM10"].rolling(window = 24, min_periods = 16).mean().values
df["PM25_24hr_avg"] = df.groupby("Station")["PM25"].rolling(window = 24, min_periods = 16).mean().values
df["SO2_24hr_avg"] = df.groupby("Station")["SO2"].rolling(window = 24, min_periods = 16).mean().values
df["NO2_24hr_avg"] = df.groupby("Station")["NO2"].rolling(window = 24, min_periods = 16).mean().values
df["NH3_24hr_avg"] = df.groupby("Station")["NH3"].rolling(window = 24, min_periods = 16).mean().values
df["CO_8hr_max"] = df.groupby("Station")["CO"].rolling(window = 8, min_periods = 1).max().values
df["O3_8hr_max"] = df.groupby("Station")["O3"].rolling(window = 8, min_periods = 1).max().values

#### PM2.5 Sub-Index calculation

In [7]:
def get_PM25_subindex(x):
    if x <= 30:
        return x * 50 / 30
    elif x <= 60:
        return 50 + (x - 30) * 50 / 30
    elif x <= 90:
        return 100 + (x - 60) * 100 / 30
    elif x <= 120:
        return 200 + (x - 90) * 100 / 30
    elif x <= 250:
        return 300 + (x - 120) * 100 / 130
    elif x > 250:
        return 400 + (x - 250) * 100 / 130
    else:
        return 0

df["PM25_SubIndex"] = df["PM25_24hr_avg"].apply(lambda x: get_PM25_subindex(x))

#### PM10 Sub-Index calculation

In [8]:
def get_PM10_subindex(x):
    if x <= 50:
        return x
    elif x <= 100:
        return x
    elif x <= 250:
        return 100 + (x - 100) * 100 / 150
    elif x <= 350:
        return 200 + (x - 250)
    elif x <= 430:
        return 300 + (x - 350) * 100 / 80
    elif x > 430:
        return 400 + (x - 430) * 100 / 80
    else:
        return 0

df["PM10_SubIndex"] = df["PM10_24hr_avg"].apply(lambda x: get_PM10_subindex(x))

#### SO2 Sub-Index calculation

In [9]:
def get_SO2_subindex(x):
    if x <= 40:
        return x * 50 / 40
    elif x <= 80:
        return 50 + (x - 40) * 50 / 40
    elif x <= 380:
        return 100 + (x - 80) * 100 / 300
    elif x <= 800:
        return 200 + (x - 380) * 100 / 420
    elif x <= 1600:
        return 300 + (x - 800) * 100 / 800
    elif x > 1600:
        return 400 + (x - 1600) * 100 / 800
    else:
        return 0

df["SO2_SubIndex"] = df["SO2_24hr_avg"].apply(lambda x: get_SO2_subindex(x))

#### NO2 Sub-Index calculation

In [10]:
def get_NO2_subindex(x):
    if x <= 40:
        return x * 50 / 40
    elif x <= 80:
        return 50 + (x - 40) * 50 / 40
    elif x <= 180:
        return 100 + (x - 80) * 100 / 100
    elif x <= 280:
        return 200 + (x - 180) * 100 / 100
    elif x <= 400:
        return 300 + (x - 280) * 100 / 120
    elif x > 400:
        return 400 + (x - 400) * 100 / 120
    else:
        return 0

df["NO2_SubIndex"] = df["NO2_24hr_avg"].apply(lambda x: get_NO2_subindex(x))

#### NH3 Sub-Index calculation

In [11]:
def get_NH3_subindex(x):
    if x <= 200:
        return x * 50 / 200
    elif x <= 400:
        return 50 + (x - 200) * 50 / 200
    elif x <= 800:
        return 100 + (x - 400) * 100 / 400
    elif x <= 1200:
        return 200 + (x - 800) * 100 / 400
    elif x <= 1800:
        return 300 + (x - 1200) * 100 / 600
    elif x > 1800:
        return 400 + (x - 1800) * 100 / 600
    else:
        return 0

df["NH3_SubIndex"] = df["NH3_24hr_avg"].apply(lambda x: get_NH3_subindex(x))

#### CO Sub-Index calculation

In [12]:
def get_CO_subindex(x):
    if x <= 1:
        return x * 50 / 1
    elif x <= 2:
        return 50 + (x - 1) * 50 / 1
    elif x <= 10:
        return 100 + (x - 2) * 100 / 8
    elif x <= 17:
        return 200 + (x - 10) * 100 / 7
    elif x <= 34:
        return 300 + (x - 17) * 100 / 17
    elif x > 34:
        return 400 + (x - 34) * 100 / 17
    else:
        return 0

df["CO_SubIndex"] = df["CO_8hr_max"].apply(lambda x: get_CO_subindex(x))

#### O3 Sub-Index calculation

In [13]:
def get_O3_subindex(x):
    if x <= 50:
        return x * 50 / 50
    elif x <= 100:
        return 50 + (x - 50) * 50 / 50
    elif x <= 168:
        return 100 + (x - 100) * 100 / 68
    elif x <= 208:
        return 200 + (x - 168) * 100 / 40
    elif x <= 748:
        return 300 + (x - 208) * 100 / 539
    elif x > 748:
        return 400 + (x - 400) * 100 / 539
    else:
        return 0

df["O3_SubIndex"] = df["O3_8hr_max"].apply(lambda x: get_O3_subindex(x))

#### AQI bucketing

In [14]:
def get_AQI_bucket(x):
    if x <= 50:
        return "Good"
    elif x <= 100:
        return "Satisfactory"
    elif x <= 200:
        return "Moderate"
    elif x <= 300:
        return "Poor"
    elif x <= 400:
        return "Very Poor"
    elif x > 400:
        return "Severe"
    else:
        return np.nan

df["Checks"] = (df["PM25_SubIndex"] > 0).astype(int) + \
                (df["PM10_SubIndex"] > 0).astype(int) + \
                (df["SO2_SubIndex"] > 0).astype(int) + \
                (df["NO2_SubIndex"] > 0).astype(int) + \
                (df["NH3_SubIndex"] > 0).astype(int) + \
                (df["CO_SubIndex"] > 0).astype(int) + \
                (df["O3_SubIndex"] > 0).astype(int)

df["AQI"] = round(df[["PM25_SubIndex", "PM10_SubIndex", "SO2_SubIndex", "NO2_SubIndex","NH3_SubIndex", "CO_SubIndex", "O3_SubIndex"]].max(axis = 1))
df.loc[df["PM25_SubIndex"] + df["PM10_SubIndex"] <= 0, "AQI"] = np.nan
df.loc[df.Checks < 3, "AQI"] = np.nan

df["AQI_bucket"] = df["AQI"].apply(lambda x: get_AQI_bucket(x))

In [15]:
df = df[~df.AQI_bucket.isna()]

In [16]:
df.head()

,Station,Date,Time,latitude,longitude,PM25,PM10,CO,NO2,NH3,...,PM25_SubIndex,PM10_SubIndex,SO2_SubIndex,NO2_SubIndex,NH3_SubIndex,CO_SubIndex,O3_SubIndex,Checks,AQI,AQI_bucket
776127,"Adarsh Nagar, Jaipur, India",2020-11-28,10:00:00,26.9006,75.8253,32.76,37.19,0.56744,6.281221,11.40,...,139.635417,77.726250,10.708594,6.810303,2.181719,40.0545,175.926471,7,176.0,Moderate
776128,"Adarsh Nagar, Jaipur, India",2020-11-28,11:00:00,26.9006,75.8253,43.00,50.81,0.82111,13.662454,19.00,...,133.970588,76.142941,11.218382,7.414289,2.332794,41.0555,175.926471,7,176.0,Moderate
776129,"Adarsh Nagar, Jaipur, India",2020-11-28,12:00:00,26.9006,75.8253,51.04,61.88,1.04141,22.765442,25.08,...,130.424074,75.350556,11.787500,8.583317,2.551528,52.0705,175.926471,7,176.0,Moderate
776130,"Adarsh Nagar, Jaipur, India",2020-11-28,13:00:00,26.9006,75.8253,55.24,68.78,1.12152,26.224897,29.64,...,127.987719,75.004737,12.265132,9.856886,2.807237,56.0760,175.926471,7,176.0,Moderate
776131,"Adarsh Nagar, Jaipur, India",2020-11-28,14:00:00,26.9006,75.8253,58.45,73.32,1.12152,24.040817,29.89,...,126.330000,74.920500,12.605625,10.866593,3.040500,56.0760,175.926471,7,176.0,Moderate


In [17]:
df.columns

Index(['Station', 'Date', 'Time', 'latitude', 'longitude', 'PM25', 'PM10',
       'CO', 'NO2', 'NH3', 'O3', 'SO2', 'PM10_24hr_avg', 'PM25_24hr_avg',
       'SO2_24hr_avg', 'NO2_24hr_avg', 'NH3_24hr_avg', 'CO_8hr_max',
       'O3_8hr_max', 'PM25_SubIndex', 'PM10_SubIndex', 'SO2_SubIndex',
       'NO2_SubIndex', 'NH3_SubIndex', 'CO_SubIndex', 'O3_SubIndex', 'Checks',
       'AQI', 'AQI_bucket'],
      dtype='object')

In [18]:
cols = ['Station', 'Date', 'Time', 'latitude', 'longitude', 'PM25', 'PM10',
       'CO', 'NO2', 'NH3', 'O3', 'SO2', 'PM10_24hr_avg', 'PM25_24hr_avg',
       'SO2_24hr_avg', 'NO2_24hr_avg', 'NH3_24hr_avg',
       'CO_8hr_max', 'O3_8hr_max', 'PM25_SubIndex', 'PM10_SubIndex',
       'SO2_SubIndex', 'NO2_SubIndex', 'NH3_SubIndex', 'CO_SubIndex',
       'O3_SubIndex', 'Checks', 'AQI', 'AQI_bucket']

In [19]:
df[cols[5:-3]] = df[cols[5:-3]].round(2)

In [20]:
df.head()

,Station,Date,Time,latitude,longitude,PM25,PM10,CO,NO2,NH3,...,PM25_SubIndex,PM10_SubIndex,SO2_SubIndex,NO2_SubIndex,NH3_SubIndex,CO_SubIndex,O3_SubIndex,Checks,AQI,AQI_bucket
776127,"Adarsh Nagar, Jaipur, India",2020-11-28,10:00:00,26.9006,75.8253,32.76,37.19,0.57,6.28,11.40,...,139.64,77.73,10.71,6.81,2.18,40.05,175.93,7,176.0,Moderate
776128,"Adarsh Nagar, Jaipur, India",2020-11-28,11:00:00,26.9006,75.8253,43.00,50.81,0.82,13.66,19.00,...,133.97,76.14,11.22,7.41,2.33,41.06,175.93,7,176.0,Moderate
776129,"Adarsh Nagar, Jaipur, India",2020-11-28,12:00:00,26.9006,75.8253,51.04,61.88,1.04,22.77,25.08,...,130.42,75.35,11.79,8.58,2.55,52.07,175.93,7,176.0,Moderate
776130,"Adarsh Nagar, Jaipur, India",2020-11-28,13:00:00,26.9006,75.8253,55.24,68.78,1.12,26.22,29.64,...,127.99,75.00,12.27,9.86,2.81,56.08,175.93,7,176.0,Moderate
776131,"Adarsh Nagar, Jaipur, India",2020-11-28,14:00:00,26.9006,75.8253,58.45,73.32,1.12,24.04,29.89,...,126.33,74.92,12.61,10.87,3.04,56.08,175.93,7,176.0,Moderate


### Further Processing

In [21]:
temp = df[['Station', 'latitude','longitude']]
temp.head(3)

,Station,latitude,longitude
776127,"Adarsh Nagar, Jaipur, India",26.9006,75.8253
776128,"Adarsh Nagar, Jaipur, India",26.9006,75.8253
776129,"Adarsh Nagar, Jaipur, India",26.9006,75.8253


#### Getting City and State for Stations

In [22]:
temp.drop_duplicates(keep='first', inplace=True)
temp.head()

,Station,latitude,longitude
776127,"Adarsh Nagar, Jaipur, India",26.9006,75.8253
356607,"Alandur Bus Depot, Chennai, Chennai, India",12.9971,80.1915
618807,"Anand Kala Kshetram, Rajamahendravaram, India",16.9873,81.7363
1594191,"Anand Vihar, Delhi, Delhi, India",28.6459,77.3149
1489311,"Anand Vihar, Hapur, India",28.7256,77.7497


In [23]:
# Using geopy library to get the respective state names
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='aqi_project')

def locationData(city):
  # These city give wrong state, so providing the states manually.
  if city =="Brajrajnagar":
    return "Odisha"
  elif city =="Pali":
    return "Rajasthan"
  else:
  # This will get the state and city name automatically.
    location = geolocator.geocode(city)
    state = location.address
    if state.split(',')[-2].replace(' ','').isalpha() == False:
      return state.split(',')[-3].strip()
    else:
      return state.split(',')[-2].strip()

In [24]:
temp['City'] = temp['Station'].apply(lambda x: x.split(",")[-2].strip())
temp['State'] = temp['City'].apply(lambda x: locationData(x))
temp.drop(columns=['latitude','longitude'], inplace=True)  # Dropping the latitude and longitude columns

df = df.merge(temp, on = "Station")
df.head()

,Station,Date,Time,latitude,longitude,PM25,PM10,CO,NO2,NH3,...,SO2_SubIndex,NO2_SubIndex,NH3_SubIndex,CO_SubIndex,O3_SubIndex,Checks,AQI,AQI_bucket,City,State
0,"Adarsh Nagar, Jaipur, India",2020-11-28,10:00:00,26.9006,75.8253,32.76,37.19,0.57,6.28,11.40,...,10.71,6.81,2.18,40.05,175.93,7,176.0,Moderate,Jaipur,Rajasthan
1,"Adarsh Nagar, Jaipur, India",2020-11-28,11:00:00,26.9006,75.8253,43.00,50.81,0.82,13.66,19.00,...,11.22,7.41,2.33,41.06,175.93,7,176.0,Moderate,Jaipur,Rajasthan
2,"Adarsh Nagar, Jaipur, India",2020-11-28,12:00:00,26.9006,75.8253,51.04,61.88,1.04,22.77,25.08,...,11.79,8.58,2.55,52.07,175.93,7,176.0,Moderate,Jaipur,Rajasthan
3,"Adarsh Nagar, Jaipur, India",2020-11-28,13:00:00,26.9006,75.8253,55.24,68.78,1.12,26.22,29.64,...,12.27,9.86,2.81,56.08,175.93,7,176.0,Moderate,Jaipur,Rajasthan
4,"Adarsh Nagar, Jaipur, India",2020-11-28,14:00:00,26.9006,75.8253,58.45,73.32,1.12,24.04,29.89,...,12.61,10.87,3.04,56.08,175.93,7,176.0,Moderate,Jaipur,Rajasthan


In [ ]:
cols = ['Station','City','State', 'Date', 'Time','PM25', 'PM10',
       'CO', 'NO2', 'NH3', 'O3', 'SO2', 'AQI', 'AQI_bucket']

df = df[cols]
df.head()

,Station,City,State,Date,Time,PM25,PM10,CO,NO2,NH3,O3,SO2,AQI,AQI_bucket
0,"Adarsh Nagar, Jaipur, India",Jaipur,Rajasthan,2020-11-28,10:00:00,32.76,37.19,0.57,6.28,11.40,130.18,12.88,176.0,Moderate
1,"Adarsh Nagar, Jaipur, India",Jaipur,Rajasthan,2020-11-28,11:00:00,43.00,50.81,0.82,13.66,19.00,97.28,15.50,176.0,Moderate
2,"Adarsh Nagar, Jaipur, India",Jaipur,Rajasthan,2020-11-28,12:00:00,51.04,61.88,1.04,22.77,25.08,63.66,17.17,176.0,Moderate
3,"Adarsh Nagar, Jaipur, India",Jaipur,Rajasthan,2020-11-28,13:00:00,55.24,68.78,1.12,26.22,29.64,42.20,16.69,176.0,Moderate
4,"Adarsh Nagar, Jaipur, India",Jaipur,Rajasthan,2020-11-28,14:00:00,58.45,73.32,1.12,24.04,29.89,33.98,15.26,176.0,Moderate


In [25]:
cols = ['Station','City','State','latitude','longitude', 'Date', 'Time','PM25', 'PM10',
       'CO', 'NO2', 'NH3', 'O3', 'SO2', 'AQI', 'AQI_bucket']

df = df[cols]
df.head()

,Station,City,State,latitude,longitude,Date,Time,PM25,PM10,CO,NO2,NH3,O3,SO2,AQI,AQI_bucket
0,"Adarsh Nagar, Jaipur, India",Jaipur,Rajasthan,26.9006,75.8253,2020-11-28,10:00:00,32.76,37.19,0.57,6.28,11.40,130.18,12.88,176.0,Moderate
1,"Adarsh Nagar, Jaipur, India",Jaipur,Rajasthan,26.9006,75.8253,2020-11-28,11:00:00,43.00,50.81,0.82,13.66,19.00,97.28,15.50,176.0,Moderate
2,"Adarsh Nagar, Jaipur, India",Jaipur,Rajasthan,26.9006,75.8253,2020-11-28,12:00:00,51.04,61.88,1.04,22.77,25.08,63.66,17.17,176.0,Moderate
3,"Adarsh Nagar, Jaipur, India",Jaipur,Rajasthan,26.9006,75.8253,2020-11-28,13:00:00,55.24,68.78,1.12,26.22,29.64,42.20,16.69,176.0,Moderate
4,"Adarsh Nagar, Jaipur, India",Jaipur,Rajasthan,26.9006,75.8253,2020-11-28,14:00:00,58.45,73.32,1.12,24.04,29.89,33.98,15.26,176.0,Moderate


In [26]:
df.to_parquet('./AQI/2020-2022_history_data_processed.parquet.brotli',compression="brotli")